In [1]:
import requests
from bs4 import BeautifulSoup as bs 
import pandas as pd
import numpy as np
# make autocompete faster shift+tab
%config Completer.use_jedi =False

In [2]:
url = "https://www.worldometers.info/coronavirus/"
response = requests.get(url)
response

<Response [200]>

In [3]:
web_page = response.content
soup = bs(web_page)
#soup

In [4]:
table = soup.find("table", attrs = {"id":"main_table_countries_today"})
#table
type(table)

bs4.element.Tag

In [5]:
# this produces an error in this webpage -> No tables found matching pattern '.+'
#dfs = pd.read_html(str(table))

# we need to make everything in between <> capital for panda to understand the table data 
import re 
patteren = r"<.+?>"
#.group()-> returns all the match objects -> .upper() -> make all of it capital
process_function = lambda x : x.group().upper()

#make a text from table soup object as re.sub needs a string 
text = str(table)

#search the text for the pattern and apply process_function on the found pattern -> make text inside <> capital
table = re.sub(patteren,process_function,text)

dfs = pd.read_html(table)
corona_table = dfs[0]
corona_table

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,NaN,North America,8576487,"+7,986",304545.0,+541,5430833.0,"+6,843",2841109.0,17748.0,NaN,NaN,NaN,NaN,NaN,North America,NaN,NaN,NaN
1,NaN,Asia,9963645,"+27,695",184741.0,+500,8250457.0,"+19,967",1528447.0,20666.0,NaN,NaN,NaN,NaN,NaN,Asia,NaN,NaN,NaN
2,NaN,South America,7759266,+628,244567.0,+34,6598969.0,+545,915730.0,15742.0,NaN,NaN,NaN,NaN,NaN,South America,NaN,NaN,NaN
3,NaN,Europe,4682130,"+19,729",218987.0,+282,2456885.0,"+9,753",2006258.0,7792.0,NaN,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN
4,NaN,Africa,1444807,+188,34760.0,+2,1190844.0,+434,219203.0,1451.0,NaN,NaN,NaN,NaN,NaN,Africa,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,NaN,Total:,4682130,"+19,729",218987.0,+282,2456885.0,"+9,753",2006258.0,7792.0,NaN,NaN,NaN,NaN,NaN,Europe,NaN,NaN,NaN
227,NaN,Total:,1444807,+188,34760.0,+2,1190844.0,+434,219203.0,1451.0,NaN,NaN,NaN,NaN,NaN,Africa,NaN,NaN,NaN
228,NaN,Total:,30887,+19,908.0,+8,28074.0,+82,1905.0,19.0,NaN,NaN,NaN,NaN,NaN,Australia/Oceania,NaN,NaN,NaN
229,NaN,Total:,721,NaN,15.0,NaN,651.0,NaN,55.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### <font color = "red">**regex explained**

In [6]:
sample_text = '<table>Banana</table>'

match = re.search(r'<.+>',sample_text)
print(match,end= '\n\n***\n')

# ? makes the + non greedy 
match = re.search(r'<.+?>',sample_text)
print(match)

print('\n***')
print(match.group().upper())

#using lambda on the match object 
re.sub(r'<.+?>',lambda x:x.group().upper(),sample_text)


<re.Match object; span=(0, 21), match='<table>Banana</table>'>

***
<re.Match object; span=(0, 7), match='<table>'>

***
<TABLE>


'<TABLE>Banana</TABLE>'

In [7]:
corona_table.TotalCases.sum()

162289715

In [8]:
# REGEX = TRUE

corona_table['NewCases'].replace(r'\+|,','' , inplace = True, regex = True)
corona_table['NewCases'] = pd.to_numeric(corona_table['NewCases'])

corona_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   #                   215 non-null    float64
 1   Country,Other       230 non-null    object 
 2   TotalCases          231 non-null    int64  
 3   NewCases            75 non-null     float64
 4   TotalDeaths         209 non-null    float64
 5   NewDeaths           52 non-null     object 
 6   TotalRecovered      226 non-null    float64
 7   NewRecovered        69 non-null     object 
 8   ActiveCases         227 non-null    float64
 9   Serious,Critical    144 non-null    float64
 10  Tot Cases/1M pop    215 non-null    float64
 11  Deaths/1M pop       193 non-null    float64
 12  TotalTests          194 non-null    float64
 13  Tests/ 1M pop       194 non-null    float64
 14  Population          213 non-null    float64
 15  Continent           227 non-null    object 
 16  1 Caseev

In [9]:
corona_table.NewRecovered.replace(r'\+|,','' , inplace = True, regex = True)
corona_table['NewRecovered'] = pd.to_numeric(corona_table['NewRecovered'])

corona_table.NewDeaths.replace(r'\+|,','' , inplace = True, regex = True)
corona_table['NewDeaths'] = pd.to_numeric(corona_table['NewDeaths'])


corona_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   #                   215 non-null    float64
 1   Country,Other       230 non-null    object 
 2   TotalCases          231 non-null    int64  
 3   NewCases            75 non-null     float64
 4   TotalDeaths         209 non-null    float64
 5   NewDeaths           52 non-null     float64
 6   TotalRecovered      226 non-null    float64
 7   NewRecovered        69 non-null     float64
 8   ActiveCases         227 non-null    float64
 9   Serious,Critical    144 non-null    float64
 10  Tot Cases/1M pop    215 non-null    float64
 11  Deaths/1M pop       193 non-null    float64
 12  TotalTests          194 non-null    float64
 13  Tests/ 1M pop       194 non-null    float64
 14  Population          213 non-null    float64
 15  Continent           227 non-null    object 
 16  1 Caseev

In [10]:
#rename the column 
corona_table.rename(columns={'Country,Other':'Country'},inplace= True)

#make a new corona table where the country name is not = to Total
corona_table = corona_table[corona_table['Country'] != 'Total:']

#drop column #
corona_table.drop(columns='#',inplace = True)

# drop rows using indexes
corona_table.drop(corona_table.index[0:8],inplace = True)

#set index to the country row 
corona_table.set_index('Country')


C:\Anacoda\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
Country,,,,,,,,,,,,,,,,,
USA,7187179,1708.0,207555.0,17.0,4438906.0,1331.0,2540718.0,14156.0,21684.0,626.0,101579471.0,306463.0,3.314575e+08,North America,46.0,1597.0,3.0
India,5823060,6957.0,92347.0,30.0,4756164.0,3173.0,974549.0,8944.0,4210.0,67.0,68928440.0,49834.0,1.383160e+09,Asia,238.0,14978.0,20.0
Brazil,4659909,NaN,139883.0,NaN,4023789.0,NaN,496237.0,8318.0,21886.0,657.0,17900000.0,84072.0,2.129127e+08,South America,46.0,1522.0,12.0
Russia,1136048,7212.0,20056.0,108.0,934146.0,4317.0,181846.0,2300.0,7784.0,137.0,44300000.0,303530.0,1.459493e+08,Europe,128.0,7277.0,3.0
Colombia,790823,NaN,24924.0,NaN,674961.0,NaN,90938.0,863.0,15504.0,489.0,3534650.0,69295.0,5.100883e+07,South America,65.0,2047.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vatican City,12,NaN,NaN,NaN,12.0,NaN,0.0,NaN,14981.0,NaN,NaN,NaN,8.010000e+02,Europe,67.0,NaN,NaN
Western Sahara,10,NaN,1.0,NaN,8.0,NaN,1.0,NaN,17.0,2.0,NaN,NaN,6.006540e+05,Africa,60065.0,600654.0,NaN
MS Zaandam,9,NaN,2.0,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
corona_table.info()
corona_table.columns

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215 entries, 8 to 222
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Country             215 non-null    object 
 1   TotalCases          215 non-null    int64  
 2   NewCases            61 non-null     float64
 3   TotalDeaths         193 non-null    float64
 4   NewDeaths           38 non-null     float64
 5   TotalRecovered      210 non-null    float64
 6   NewRecovered        55 non-null     float64
 7   ActiveCases         211 non-null    float64
 8   Serious,Critical    128 non-null    float64
 9   Tot Cases/1M pop    213 non-null    float64
 10  Deaths/1M pop       191 non-null    float64
 11  TotalTests          194 non-null    float64
 12  Tests/ 1M pop       194 non-null    float64
 13  Population          213 non-null    float64
 14  Continent           213 non-null    object 
 15  1 Caseevery X ppl   213 non-null    float64
 16  1 Deathe

Index(['Country', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/ 1M pop',
       'Population', 'Continent', '1 Caseevery X ppl', '1 Deathevery X ppl',
       '1 Testevery X ppl'],
      dtype='object')

In [ ]:
# or use this to find all the rows we want to drop 

#rows_to_drop=corona_table.drop(corona_table[corona_table["Country"].isin(["Total:","Africa","Asia","North America","South America","Europe","Oceania","World"])].index, inplace=True)